In [1]:
from music21.midi import MidiFile
from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#rock
path = "/content/gdrive/My Drive/EE239AS/freemidi/rock/rock_midi"
output_path = "/content/gdrive/My Drive/EE239AS/freemidi/rock/processed/rock_midi"

In [0]:
#edm
path = "/content/gdrive/My Drive/EE239AS/freemidi/edm/edm_midi"
output_path = "/content/gdrive/My Drive/EE239AS/freemidi/edm/processed/edm_midi"

In [0]:
import os
from music21 import converter, instrument, note, chord, stream
from music21.midi import MidiException


def heuristics(part):
  pitches = {}
  mode_pitch = None  

  for pitch in part.pitches:
    if mode_pitch is None:
      mode_pitch = pitch
    try:
      pitches[pitch] += 1
    except KeyError:
      pitches[pitch] = 1
    if pitches[pitch] > pitches[mode_pitch]:
      mode_pitch = pitch

  num_pitches = 0
  mean_pitch = 0
  for pitch in pitches:
    num_pitches += pitches[pitch]
    mean_pitch += pitch.frequency * pitches[pitch]
  
  mean_pitch /= num_pitches
  
  pitch_variance = 0
  for pitch in pitches:
    pitch_variance += (pitch.frequency-mean_pitch)**2
    
  return [part.duration.quarterLength, num_pitches, mean_pitch, pitch_variance]


In [9]:
from music21.midi.translate import streamToMidiFile

files = os.listdir(path)

for file in files:
  print(file)
  try:
    midi = converter.parse(os.path.join(path, file))
  except MidiException:
    continue
  ps = midi.parts.stream()

  parts = {}
  weights = [.25, .25, .1, .4]
  melody_part = None

  for p in ps:
    if melody_part is None:
      melody_part = p

    hers = heuristics(p)
#     print(p, hers)
    parts[p] = sum([hers[i]*weights[i] for i in range(len(hers))])

    if parts[p] > parts[melody_part]:
      melody_part = p


#   print(melody_part, parts[melody_part])
#   print(parts)

  mf = streamToMidiFile(melody_part)
  mf.open(os.path.join(output_path, file), "wb")
  mf.write()
  mf.close()

Alan Walker - Alone  (midi by Carlo Prato) (www.cprato.com).mid
Alan Walker - Faded (Original Mix) (midi by Carlo Prato) (www.cprato.com).mid
Alan Walker - Sing Me To Sleep  (midi by Carlo Prato) (www.cprato.com).mid
Alan Walker - Tired  (midi by Carlo Prato) (www.cprato.com).mid
Bruno Mars - That's What I Like (Alan Walker Remix) (midi by Carlo Prato) (www.cprato.com).mid
Avicii - Dear Boy.mid
Avicii - Without You.mid
Avicii - Lonely Together.mid
The Chainsmokers feat. XYLO - Setting Fires.mid
The Chainsmokers - Paris.mid
Galantis - Love On Me  (midi by Carlo Prato) (www.cprato.com).mid
The Chainsmokers & Coldplay - Something Just Like This.mid
The Chainsmokers ft. Phoebe Ryan - All We Know.mid
The Chainsmokers Ft. Daya - Don't Let Me Down.mid
The Chainsmokers ft. Halsey - Closer.mid
Galantis - No Money  (midi by Carlo Prato) (www.cprato.com).mid
Marshmello - Ritual (Original Mix) (midi by Carlo Prato) (www.cprato.com).mid
Marshmello - Alone (Original Mix) (midi by Carlo Prato) (www.c

In [0]:
files = os.listdir(output_path)
for f in files:
  try:
    midi = converter.parse(os.path.join(output_path, f))
  except IndexError as e:
    print(f, e)
    os.remove(os.path.join(output_path, f))
  